In [5]:
#| hide
from fastsqlmodel.db import *

# FastSQLModel

> Simplie and Fast utility for SQLModel/SQLAlchemy and Alembic


## Overview

FastSQLModel is a utility for simplifying the process of using SQLModel/SQLAlchemy and Alembic. It provides a CLI for initializing and managing Alembic migrations, and a set of tools for working with SQLModel and SQLAlchemy models.

## Features

- **CLI for Alembic**: FastSQLModel provides a CLI for initializing and managing Alembic migrations.
- **SQLModel and SQLAlchemy Models**: FastSQLModel provides a set of tools for working with SQLModel and SQLAlchemy models.
- **Database Management**: FastSQLModel provides a set of tools for managing the database, including creating, dropping, and querying the database.


## Developer Guide

### Install FastSQLModel in Development 
If you want to make changes to the package, you can install it in development mode.
This project uses nbdev for development, so you can make changes to the code and documentation in the notebooks under the nbs/ directory.
To find out more about amazing nbdev, visit the [nbdev documentation](https://nbdev.fast.ai/index.html).

To make changes to the package, you can install it in development mode.
```sh
# make sure FastSQLModel package is installed in development mode
$ pip install -e .

# make changes under nbs/ directory
# ...

# compile to have changes apply to FastSQLModel
$ nbdev_prepare
```

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/ndendic/FastSQLModel.git
```

or from [conda][conda]

```sh
$ conda install -c ndendic FastSQLModel
```

or from [pypi][pypi]


```sh
$ pip install FastSQLModel
```


[repo]: https://github.com/ndendic/FastSQLModel
[docs]: https://ndendic.github.io/FastSQLModel/
[pypi]: https://pypi.org/project/FastSQLModel/
[conda]: https://anaconda.org/ndendic/FastSQLModel

To establish a connection to the database, please specify the `DATABASE_URL` in the `.env` file.

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/ndendic/FastSQLModel
[docs]: https://ndendic.github.io/FastSQLModel/
[pypi]: https://pypi.org/project/FastSQLModel/
[conda]: https://anaconda.org/ndendic/FastSQLModel

## How to use

### Create your first model

To create your first model, you can can import the BaseTable class from the fastsqlmodel.db module and create a new model by subclassing it.
BaseTable is a subclass of SQLModel, so it has all the same features, but it also has a few extra features to help with some standard db operations and 3 extra fields: 
 - id: primary key, default to a uuid4
 - created_at: datetime, default to now
 - updated_at: datetime, default to now, and updated on every save

```python
class BaseTable(SQLModel):
    model_config = ConfigDict(json_encoders={datetime: lambda dt: dt.isoformat()})
    id: UUID = Field(default_factory=uuid4, primary_key=True)
    created_at: datetime = Field(
        default_factory=utc_now,
        sa_type= sa.DateTime(timezone=True),
        sa_column_kwargs={"server_default": sa.func.now()},
        nullable=False,
        title="Created At",
        schema_extra={"icon": "clock", "input_type": "datetime"},
    )
    updated_at: datetime = Field(
        default_factory=utc_now,
        sa_type=sa.DateTime(timezone=True),
        sa_column_kwargs={
            "server_default": sa.func.now(),
            "server_onupdate": sa.func.now(),
        },
        # onupdate=utc_now,
        nullable=False,
        title="Updated At",
        schema_extra={"icon": "clock", "input_type": "datetime"},
    )
```



Here is an example of how to create a new model using BaseTable

```python
# users.py
from typing import Optional
from sqlmodel import Field
from datetime import datetime
from fastsqlmodel.db import BaseTable

class User(BaseTable, table=True):
    name: Optional[str] = Field(nullable=True)
    email: str = Field(nullable=False)
    password: str = Field(nullable=False)
    joined_at: datetime = Field(nullable=False)
```



Now that you have created your first model, you can use the CLI to initialize and manage Alembic project.

```sh
$ fastmodel init
```
This will create a new Alembic project in the current directory, and create a new .alembic.ini file.

2. Then make sure to add your models to the migrations/env.py file before running migrations.

```python
# migrations/env.py
from users import User
# ...
```

3. Now you can run migrations to prepare the database for your models.

```sh
$ fastmodel migrations
```

4. And now you can migrate your models to the database.

```sh
$ fastmodel migrate
```


Let's see how this works

In [14]:
!fastmodel init

Successfully initialized Alembic in migrations directory!
Please make sure to add your models to migrations/env.py file before running 
migrations!


In [19]:
!fastmodel migrations

Generating Alembic migration with message: Pushing changes
DATABASE_URL sqlite:///test.db
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.autogenerate.compare] Detected added table 'user'
  Generating /home/ndendic/WebDev/FastSQLModel/nbs/migrations/versions/5289002d2
  4df_pushing_changes.py ...  done
Migration created successfully!


In [20]:
!fastmodel migrate

Applying database migrations...
DATABASE_URL sqlite:///test.db
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade a035a7116947 -> 5289002d24df, Pushing changes
Migrations applied successfully!


In [1]:
#| hide
from users import User
from datetime import datetime

In [3]:
user = User(name="Homer Simpson", email="homer@simpson.com", password="password", joined_at=datetime.now())
user.save()
user.model_dump()


{'name': 'Homer Simpson',
 'email': 'homer@simpson.com',
 'password': 'password',
 'joined_at': datetime.datetime(2024, 12, 18, 14, 51, 33, 62687),
 'id': UUID('10dc919b-c7de-4453-8a3a-63c4ffd7da4d'),
 'created_at': datetime.datetime(2024, 12, 18, 13, 51, 33, 65307, tzinfo=datetime.timezone.utc),
 'updated_at': datetime.datetime(2024, 12, 18, 13, 51, 33, 65334, tzinfo=datetime.timezone.utc)}

In [8]:
homer = User.get(id="10dc919b-c7de-4453-8a3a-63c4ffd7da4d")
homer.name, homer.email


('Homer Simpson', 'homer@simpson.com')

In [7]:
homer = User.get("homer@simpson.com",alt_key="email")
homer.name



'Homer Simpson'

In [10]:
homer.email = "homer.simpson@simpson.com"
homer.save()
homer = User.get("homer.simpson@simpson.com",alt_key="email")
homer.email


'homer.simpson@simpson.com'

In [11]:
bart = User.get("bart@simpson.com",alt_key="email")
bart.name



'Bart Simpson'

In [17]:
User.all()

[User(id=UUID('3b04291b-347f-4edf-bcbe-20dc70f5df42'), updated_at=datetime.datetime(2024, 12, 18, 13, 40, 1, 620576), created_at=datetime.datetime(2024, 12, 18, 13, 40, 1, 620566), password='password', name='John Doe', email='john.doe@example.com', joined_at=datetime.datetime(2024, 12, 18, 14, 40, 1, 601127)),
 User(id=UUID('dea8e467-89dc-42f9-b623-821cf72c5ecd'), updated_at=datetime.datetime(2024, 12, 18, 13, 40, 41, 648055), created_at=datetime.datetime(2024, 12, 18, 13, 40, 41, 648046), password='password', name='John Doe', email='john.doe@example.com', joined_at=datetime.datetime(2024, 12, 18, 14, 40, 41, 637238)),
 User(id=UUID('c9c7ab75-9ffe-4688-8e81-2b8f0c03398a'), updated_at=datetime.datetime(2024, 12, 18, 13, 40, 45, 100049), created_at=datetime.datetime(2024, 12, 18, 13, 40, 45, 100034), password='password', name='John Doe', email='john.doe@example.com', joined_at=datetime.datetime(2024, 12, 18, 14, 40, 45, 89970)),
 User(id=UUID('0626af07-521a-49a9-bfda-1f78791909a0'), upda

In [2]:
users = User.search(search_value="Homer")
for user in users:
    print(f"Name: {user.name} , Email: {user.email}, ID: {user.id}")
users

Name: Homer Simpson , Email: homer.simpson@simpson.com, ID: 10dc919b-c7de-4453-8a3a-63c4ffd7da4d


[User(email='homer.simpson@simpson.com', name='Homer Simpson', joined_at=datetime.datetime(2024, 12, 18, 14, 51, 33, 62687), updated_at=datetime.datetime(2024, 12, 18, 13, 54, 4, 230616), id=UUID('10dc919b-c7de-4453-8a3a-63c4ffd7da4d'), created_at=datetime.datetime(2024, 12, 18, 13, 51, 33, 76743), password='password')]

In [3]:
users = User.search(search_value="Simpson", fields=['name','email'])
users


[('Homer Simpson', 'homer.simpson@simpson.com'),
 ('Bart Simpson', 'bart@simpson.com')]

In [3]:
users = User.search(search_value="john")
users

[User(updated_at=datetime.datetime(2024, 12, 18, 13, 41, 55, 872051), id=UUID('0626af07-521a-49a9-bfda-1f78791909a0'), created_at=datetime.datetime(2024, 12, 18, 13, 41, 55, 872034), password='password', email='john.doe@example.com', name='John Doe', joined_at=datetime.datetime(2024, 12, 18, 14, 41, 55, 861727)),
 User(updated_at=datetime.datetime(2024, 12, 18, 13, 40, 1, 620576), id=UUID('3b04291b-347f-4edf-bcbe-20dc70f5df42'), created_at=datetime.datetime(2024, 12, 18, 13, 40, 1, 620566), password='password', email='john.doe@example.com', name='John Doe', joined_at=datetime.datetime(2024, 12, 18, 14, 40, 1, 601127)),
 User(updated_at=datetime.datetime(2024, 12, 18, 13, 40, 45, 100049), id=UUID('c9c7ab75-9ffe-4688-8e81-2b8f0c03398a'), created_at=datetime.datetime(2024, 12, 18, 13, 40, 45, 100034), password='password', email='john.doe@example.com', name='John Doe', joined_at=datetime.datetime(2024, 12, 18, 14, 40, 45, 89970)),
 User(updated_at=datetime.datetime(2024, 12, 18, 13, 40, 41

In [7]:
for user in users:
    user.delete()

for user in User.all():
    print(user.name)


Bart Simpson
Homer Simpson


In [2]:
results = User.filter(name="Homer",exact_match=False)
results


[]